In [1]:
import pandas as pd
from emoji import UNICODE_EMOJI
import emoji
import regex as re
import tweepy
from tqdm import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import lightgbm as lgbm
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import pickle
# loading the tensorflow universal sentence encoder 4 model
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed = hub.load("universal-sentence-encoder_4")

2021-12-07 16:15:50.828815: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# setup API auhorizaion

# assign the values accordingly
consumer_key = "jfpVhd1zOxM6uqR1GjcQzEsns"
consumer_secret = "c4PdCb2fvIbFRHlgQJlT6k0cPlRUtl1E0akP5dSvmKz1NFUHmr"
access_token = "1250984376671670272-3wFUr43oEV9NUiPtv8f7ygVT74Rw3s"
access_token_secret = "2OyDgcbZWYbVwgvFJttSIEroJzmOOTwq9hGMGmVeRs9mg"

# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [3]:
# this is how you construct a Prediction Set. Provide a list of user ids and the function will build your dataset
# save directory is where you would like your dataset to be saved to. 
# ex: r'/Users/lincolnrychecky/desktop/trainingDataBots.csv' saves the file as a csv named trainingDataBots.csv to my desktop

def build_prediction_set(userIds, saveDirectory):
    # construct the training set by calling predefined functions
    for id in tqdm(userIds):
        # Put in a try-except block in case a forbidden user id is in there
        try:
            print(id)
            user = api.get_user(user_id=id)

            recentFiveTweets = api.user_timeline(user_id=id, 
                                    # 200 is the maximum allowed count
                                    count=11,
                                    include_rts = True,
                                    # Necessary to keep full_text 
                                    # otherwise only the first 140 words are extracted
                                    tweet_mode = 'extended'
                                    )
            fullText = ""
            for info in recentFiveTweets:
                fullText = fullText + info.full_text + " "
            cleanedText = clean(fullText)

            ids.append(id)
            emojis.append(split_count(fullText))
            followers.append(getFollowers(user))
            following.append(getFriends(user))
            hashtags.append(countHashtags(fullText))
            urls.append(countUrls(fullText))
            statuses.append(getStatuses(user))
            verified.append(getVerified(user))
            text.append(cleanedText)
            rawText.append(fullText)
            human.append(0)
        except:
            pass
        
    # turn dict into dataframe for export
    trainDataBot = pd.DataFrame.from_dict(dataDict)
    trainDataBot.to_csv(saveDirectory, index = False)